# Домашнее задание №4 - Линейные модели - Часть 1

Заполните ответы в [форму](https://docs.google.com/forms/d/e/1FAIpQLSe5SEqn2yiQtKlTHkrifwTgWRH8asbCzCkL_uRl70vHxl46Ng/viewform)

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(palette='deep', style='darkgrid', rc={"figure.figsize": (15, 4)})
import scipy.stats as st

import warnings
warnings.simplefilter('ignore')

Линейные модели отлично (и лучше) подходят для работы данным в которых очень много признаков, обычно они могут представлять из себя достаточно разряженные признаки.
Поэтому воспользуемся датасетом [fetch_20newsgroups](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.svm import LinearSVC

В данном датасете 20 типов новостей, однако мы будем решать задачу бинарной классификации и поэтому выберем только две категории

In [ ]:
easy_categories = ['alt.atheism', 'sci.space'] # Очень разные категории 
hard_categories = ['rec.sport.baseball', 'rec.sport.hockey'] # Очень близкие категории

In [ ]:
two_groups_data = fetch_20newsgroups(subset='all', 
                                     categories=easy_categories,
                                     remove=('headers', 'footers', 'quotes'))

Поделим датасет на тестовую и валидационную выборку:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(two_groups_data.data, 
                                                    two_groups_data.target, 
                                                    test_size=0.30, random_state=18)

Посмотрим примеры текстов из разных категорий

In [ ]:
n = 832
print('Текст из категории ', easy_categories[y_train[n]], ':')
print('='*50)
print(x_train[n])

In [ ]:
n = 124
print('Текст из категории ', easy_categories[y_train[n]], ':')
print('='*50)
print(x_train[n])

Для работы с текстами, в машинном обучению применяется метод Мешка слов. Обязательно прочитайте про методы векторизации текста например [здесь](https://machinelearningmastery.com/gentle-introduction-bag-words-model/)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Зададим параметры для CountVectorizer. Определим Максимальное колличество признаков равным 500

In [ ]:
CV = CountVectorizer(max_features=500)

Применим векторайзер, учитывайте, что для метода существуют не только фунции .fit и .transform, но и .fit_transform

In [ ]:
x_train_vectorized = # Ваш код здесь
x_test_vectorized = # Ваш код здесь

In [ ]:
x_train_vectorized.shape

Обучим логистическую регрессию с параметрами по умолчанию:

In [ ]:
logit = LogisticRegression()
# Ваш код здесь

__ Задание 1:__ Чему равны accuracy score и roc auc score на тестовой выборке?   
> Будьте внимательны, что для вычисления roc_auc Вам нужно использовать .predict_proba

In [ ]:
# Ваш код здесь

Одно из Важных свойст линейных моделей - интерпретируемость результатов. Пожалуйста, изучите подробно этот [материал]('https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf')

__ Задание 2:__ Отсортируйте в порядке уменьшения важности, признаки для категории новостей "Космос"

Но мы будем строить график важности для обоих признаков.

In [ ]:
feat_imp = pd.DataFrame('Ваш код здесь', index='Ваш код здесь',
                        columns=['feature importance']
                                          ).sort_values(by='feature importance', ascending=False)
feat_imp = pd.concat([feat_imp.head(20), feat_imp.tail(20)])

colors = ["red" if c < 0 else "blue" for c in feat_imp.values]

plt.bar(feat_imp.index, feat_imp['feature importance'], align='center', color=colors)
plt.xticks(feat_imp.index, rotation=90)
plt.ylabel('Feature importance')
plt.title('Важность признаков')
plt.show()

# Кривые обучения и Валидации

При обучении моделей, очень Важно строить кривые обучения и валидации, они помогают определить:
1. Достаточно ли нам данных? 
2. Существует ли Тендеция к переобучению? 
3. Наблюдается ли недообучение? 

Подробнее [Открытый курс Машинного обучения: Тема 5](https://habrahabr.ru/company/ods/blog/323890/#5-krivye-validacii-i-obucheniya)

In [ ]:
def curve(acc_train, acc_test):
    plt.plot(acc_train, label='train')
    plt.plot(acc_test, label='test')
    plt.legend()
    plt.show()

Построим их:

In [ ]:
acc_test = []
acc_train = []

sgd = SGDClassifier(loss='hinge', alpha=0.1, max_iter=100, random_state=18)

shapes = 50
for i in range(120):
    shapes = shapes+10
    sgd.fit(x_train_vectorized[:shapes], y_train[:shapes] )
    acc_train = np.append(acc_train, accuracy_score(y_train[:shapes], sgd.predict(x_train_vectorized)[:shapes]))    
    acc_test = np.append(acc_test, accuracy_score(y_test, sgd.predict(x_test_vectorized)))

curve(acc_train, acc_test)

In [ ]:
acc_test = []
acc_train = []

c = np.linspace(1e-18, 10)

for i in c:
    sgd = SGDClassifier(loss='hinge', alpha=i, max_iter=100, random_state=18)
    sgd.fit(x_train_vectorized, y_train)
    acc_train = np.append(acc_train, accuracy_score(y_train, sgd.predict(x_train_vectorized)))    
    acc_test = np.append(acc_test, accuracy_score(y_test, sgd.predict(x_test_vectorized)))
    
curve(acc_train, acc_test)

__Задание 3__: Можно ли однозначно сказать, что данных достаточно? (При этих параметрах)  
__Задание 4__: Наблюдается ли переобучение?  (При этих параметрах)  

Попробуйте посмотреть как будут меяться кривые обучения и валидации при различных параметрах.
Попробуйте так же применить все те же манипуляции к выборке навастей со сложными (близкими категориями)

# Часть 2: Решение Многоклассовой задачи

In [ ]:
two_groups_data = fetch_20newsgroups(subset='all', 
                                     remove=('headers', 'footers', 'quotes'))

x_train, x_test, y_train, y_test = train_test_split(two_groups_data.data, 
                                                    two_groups_data.target, 
                                                    test_size=0.30, random_state=18)

In [ ]:
CV = CountVectorizer(max_features=20000)
x_train_vectorized = CV.fit_transform(x_train)
x_test_vectorized = CV.transform(x_test)

In [ ]:
sgd = SGDClassifier(alpha=1e-20, n_jobs=-1, random_state=124)
sgd.fit(x_train_vectorized, y_train) 
accuracy_score(y_test, sgd.predict(x_test_vectorized))

Итак, у нас есть Baseline - 0.61231.   

__Ваша задача:__
1. Добиться Увеличения бейзлайна любыми известными вам способами (но см. условия)
2. Загрузить в форму Ваш ноутбук с решением

В зависимости от итоговой метрики, договоримся как распределим баллы.  

Условия:
- Чтобы было честно, нейронки пока не используем
- Должна быть воспроизводимость результатов. Не забывайте фиксировать random seed, если воспроизводимости не будет, то результатом будет то значение метрики, которое получилось у меня при воспроизведении Вашего ноутбука. 
 
